## Task 8:
* Perform a linear regression between the output variable diff2 co2 and the set of predictors:
annual harmonics from Task 6 and semi-annual periodic components (cos(2*pi*months/6)
and sin(2*pi*months/6), with months=rep(1:12, 39)[-c(1,2)]).
* Analyze the residuals (time series plot, histogram and ACF) and fitted values (time series
and qq-plot).
* Are there any remaining temporal components left?

## 8-1 linear regression for annaul and semi annual  harmonic  predictors 

In [ ]:
# Task 8: Annual + Semiannual Harmonic Regression
# This is a standalone notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf
from statsmodels.stats.diagnostic import acorr_ljungbox

sns.set_style("whitegrid")

print("=" * 70)
print("Task 8: Annual + Semiannual Harmonic Regression")
print("=" * 70)

# ========== Step 1: Data Loading ==========
print("\n[Step 1] Data Loading")

df = pd.read_csv('/Users/lihong/Desktop/ATS_IndividualProjTwo/co2.csv')

def decimal_year_to_date(decimal_year):
    year = int(decimal_year)
    remainder = decimal_year - year
    year_start = pd.Timestamp(year=year, month=1, day=1)
    year_end = pd.Timestamp(year=year+1, month=1, day=1)
    return year_start + (year_end - year_start) * remainder

df['date'] = df['time(co2)'].apply(decimal_year_to_date)
df['co2'] = pd.to_numeric(df['co2'], errors='coerce')
df.set_index('date', inplace=True)
df = df.dropna()

print("✓ Data loaded")
print(f"  Sample size: {len(df)}")

# ========== Step 2: Create diff2_co2 ==========
print("\n" + "=" * 70)
print("[Step 2] Create diff2_co2")
print("=" * 70)

df['diff1_co2'] = df['co2'].diff(periods=1)
df['diff2_co2'] = df['diff1_co2'].diff(periods=1)

print("✓ diff2_co2 computed")
print(f"  Valid observations: {df['diff2_co2'].notna().sum()}")

# ========== Step 3: Create Harmonic Variables ==========
print("\n" + "=" * 70)
print("[Step 3] Create Annual and Semiannual Harmonics")
print("=" * 70)

print("""
[Extended Harmonic Model]

We now use FOUR harmonic terms to model seasonality:

1) Annual harmonics (12-month cycle):
   - cos(2π × month / 12)
   - sin(2π × month / 12)

2) Semiannual harmonics (6-month cycle):
   - cos(2π × month / 6)
   - sin(2π × month / 6)

Regression:
  diff2_co2 = β₀ + β₁·cos(2πt/12) + β₂·sin(2πt/12)
                  + β₃·cos(2πt/6)  + β₄·sin(2πt/6) + ε

Why add the semiannual cycle?
- In Task 6, the ACF of residuals had notable peaks at lags 6, 18, 30...
- That suggests a 6-month component in addition to the 12-month cycle.
- Adding semiannual harmonics helps capture this extra periodicity.
""")

# Month sequence (skip first two months due to two differences in diff2_co2)
n_total = len(df)
n_years = int(np.ceil(n_total / 12))
months_full = np.tile(np.arange(1, 13), n_years)[:n_total]
months = months_full[2:]  # drop first two (diff2_co2 are NaN there)

# Annual harmonics (12-month)
cos_annual = np.cos(2 * np.pi * months / 12)
sin_annual = np.sin(2 * np.pi * months / 12)

# Semiannual harmonics (6-month)
cos_semi = np.cos(2 * np.pi * months / 6)
sin_semi = np.sin(2 * np.pi * months / 6)

# Add to dataframe
df['cos_annual'] = np.nan
df['sin_annual'] = np.nan
df['cos_semi'] = np.nan
df['sin_semi'] = np.nan

df.iloc[2:, df.columns.get_loc('cos_annual')] = cos_annual
df.iloc[2:, df.columns.get_loc('sin_annual')] = sin_annual
df.iloc[2:, df.columns.get_loc('cos_semi')] = cos_semi
df.iloc[2:, df.columns.get_loc('sin_semi')] = sin_semi

print("\n✓ Harmonic variables created")
print(f"  Annual cos/sin valid values: {df['cos_annual'].notna().sum()}")
print(f"  Semiannual cos/sin valid values: {df['cos_semi'].notna().sum()}")

# ========== Step 4: Harmonic Regression ==========
print("\n" + "=" * 70)
print("[Step 4] Fit Extended Harmonic Regression")
print("=" * 70)

# Prepare regression data
valid_mask = (df['diff2_co2'].notna() &
              df['cos_annual'].notna() &
              df['sin_annual'].notna() &
              df['cos_semi'].notna() &
              df['sin_semi'].notna())

df_valid = df[valid_mask].copy()

# Regressors: four harmonic terms
X = df_valid[['cos_annual', 'sin_annual', 'cos_semi', 'sin_semi']].values
y = df_valid['diff2_co2'].values

# Add intercept
X_with_const = sm.add_constant(X)

# OLS
model = sm.OLS(y, X_with_const)
results = model.fit()

# Save fitted values and residuals
df.loc[valid_mask, 'task8_fitted'] = results.fittedvalues
df.loc[valid_mask, 'task8_residuals'] = results.resid

print("\n✓ Regression completed")
print(f"  Effective sample size: {len(df_valid)}")

# ========== Step 5: Regression Summary ==========
print("\n" + "=" * 70)
print("[Step 5] Regression Summary")
print("=" * 70)

print("\nFull regression summary:")
print(results.summary())

print("\n[Regression equation]")
print(f"  diff2_co2 = {results.params[0]:.6f}")
print(f"            + {results.params[1]:.6f} × cos(2πt/12)")
print(f"            + {results.params[2]:.6f} × sin(2πt/12)")
print(f"            + {results.params[3]:.6f} × cos(2πt/6)")
print(f"            + {results.params[4]:.6f} × sin(2πt/6)")

print("\n[Goodness of fit]")
print(f"  R²:                {results.rsquared:.6f}")
print(f"  Adjusted R²:       {results.rsquared_adj:.6f}")
print(f"  F-statistic:       {results.fvalue:.4f}")
print(f"  F-test p-value:    {results.f_pvalue:.2e}")

print("\n[Coefficient significance]")
coef_names = ['Intercept', 'cos(2πt/12)', 'sin(2πt/12)', 'cos(2πt/6)', 'sin(2πt/6)']
for i, name in enumerate(coef_names):
    sig = "***" if results.pvalues[i] < 0.001 else "**" if results.pvalues[i] < 0.01 else "*" if results.pvalues[i] < 0.05 else ""
    print(f"  {name:15s}: coef={results.params[i]:8.5f}, t={results.tvalues[i]:7.4f}, p={results.pvalues[i]:.6f} {sig}")

residuals = df['task8_residuals'].dropna()

print("\n[Residual statistics]")
print(f"  Mean:        {residuals.mean():.8f}")
print(f"  Std:         {residuals.std():.4f}")
print(f"  Min:         {residuals.min():.4f}")
print(f"  Max:         {residuals.max():.4f}")

print("\n[Compare with Task 6 (annual harmonics only)]")
# Refit Task 6 model on the same df_valid subset for apples-to-apples comparison
X_task6 = df_valid[['cos_annual', 'sin_annual']].values
X_task6_const = sm.add_constant(X_task6)
model_task6 = sm.OLS(y, X_task6_const)
results_task6 = model_task6.fit()

print(f"  Task 6 R²:         {results_task6.rsquared:.6f}")
print(f"  Task 8 R²:         {results.rsquared:.6f}")
print(f"  R² improvement:    {results.rsquared - results_task6.rsquared:.6f}")
print(f"  Residual std drop: {results_task6.resid.std():.4f} → {residuals.std():.4f}")

# ========== Step 6: Residual Analysis (TS, Hist, ACF/PACF) ==========
print("\n" + "=" * 70)
print("[Step 6] Residual Analysis")
print("=" * 70)

fig, axes = plt.subplots(3, 2, figsize=(18, 14))

# (a) Residual time series
axes[0, 0].plot(residuals.index, residuals.values, linewidth=1, color='darkgreen', alpha=0.7)
axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0, 0].axhline(y=residuals.std(), color='blue', linestyle=':', linewidth=1.5, alpha=0.6)
axes[0, 0].axhline(y=-residuals.std(), color='blue', linestyle=':', linewidth=1.5, alpha=0.6)
axes[0, 0].fill_between(residuals.index, -residuals.std(), residuals.std(), alpha=0.2, color='blue')
axes[0, 0].set_title('(a) Residuals Time Series', fontsize=13, fontweight='bold')
axes[0, 0].set_xlabel('Year', fontsize=11)
axes[0, 0].set_ylabel('Residuals', fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# (b) Residual histogram
axes[0, 1].hist(residuals, bins=50, color='darkgreen', alpha=0.7, edgecolor='black', density=True)
mu, sigma = residuals.mean(), residuals.std()
x = np.linspace(residuals.min(), residuals.max(), 100)
axes[0, 1].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2.5)
axes[0, 1].set_title('(b) Residuals Distribution', fontsize=13, fontweight='bold')
axes[0, 1].set_xlabel('Residuals', fontsize=11)
axes[0, 1].set_ylabel('Density', fontsize=11)
axes[0, 1].grid(True, alpha=0.3, axis='y')

# (c) Q-Q plot
stats.probplot(residuals, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('(c) Q-Q Plot (Normality Check)', fontsize=13, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# (d) ACF
plot_acf(residuals, lags=48, ax=axes[1, 1], alpha=0.05)
axes[1, 1].set_title('(d) ACF of Residuals', fontsize=13, fontweight='bold')
axes[1, 1].set_xlabel('Lag', fontsize=11)
axes[1, 1].grid(True, alpha=0.3)

# (e) PACF
plot_pacf(residuals, lags=48, ax=axes[2, 0], alpha=0.05, method='ywm')
axes[2, 0].set_title('(e) PACF of Residuals', fontsize=13, fontweight='bold')
axes[2, 0].set_xlabel('Lag', fontsize=11)
axes[2, 0].grid(True, alpha=0.3)

# (f) Residuals vs Fitted
fitted = df.loc[valid_mask, 'task8_fitted']
axes[2, 1].scatter(fitted, residuals, alpha=0.5, s=30, color='darkgreen', edgecolors='black', linewidth=0.5)
axes[2, 1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[2, 1].set_title('(f) Residuals vs Fitted Values', fontsize=13, fontweight='bold')
axes[2, 1].set_xlabel('Fitted Values', fontsize=11)
axes[2, 1].set_ylabel('Residuals', fontsize=11)
axes[2, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('task8_residuals_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Residual analysis figure saved")

# ========== Step 7: Fitted Values Analysis ==========
print("\n" + "=" * 70)
print("[Step 7] Fitted Values Analysis")
print("=" * 70)

fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# (a) Original vs Fitted
original = df_valid['diff2_co2']
axes[0, 0].plot(original.index, original.values, linewidth=1.5,
               color='darkred', alpha=0.6, label='Original diff2_co2')
axes[0, 0].plot(fitted.index, fitted.values, linewidth=2.5,
               color='blue', label='Fitted Values', zorder=3)
axes[0, 0].axhline(y=0, color='black', linestyle='--', linewidth=1)
axes[0, 0].set_title('(a) Original vs Fitted Values', fontsize=13, fontweight='bold')
axes[0, 0].set_xlabel('Year', fontsize=11)
axes[0, 0].set_ylabel('diff2_co2', fontsize=11)
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# (b) Zoomed view
zoom_start = 100
zoom_end = 200
zoom_mask = (original.index >= original.index[zoom_start]) & (original.index <= original.index[min(zoom_end, len(original)-1)])
axes[0, 1].plot(original[zoom_mask].index, original[zoom_mask].values, linewidth=2,
               color='darkred', alpha=0.7, marker='o', markersize=4, label='Original')
axes[0, 1].plot(fitted[zoom_mask].index, fitted[zoom_mask].values, linewidth=3,
               color='blue', label='Fitted')
axes[0, 1].set_title('(b) Zoomed View', fontsize=13, fontweight='bold')
axes[0, 1].set_xlabel('Year', fontsize=11)
axes[0, 1].set_ylabel('diff2_co2', fontsize=11)
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# (c) Q-Q plot (original)
stats.probplot(original, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('(c) Q-Q Plot: Original diff2_co2', fontsize=13, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# (d) Fitted vs Original (Q-Q style)
axes[1, 1].scatter(fitted, original, alpha=0.5, s=30, color='purple', edgecolors='black', linewidth=0.3)
min_val = min(fitted.min(), original.min())
max_val = max(fitted.max(), original.max())
axes[1, 1].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2.5, label='Perfect Fit')
axes[1, 1].set_title('(d) Fitted vs Original (Q-Q style)', fontsize=13, fontweight='bold')
axes[1, 1].set_xlabel('Fitted Values', fontsize=11)
axes[1, 1].set_ylabel('Original Values', fontsize=11)
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

corr = np.corrcoef(fitted, original)[0, 1]
axes[1, 1].text(0.02, 0.98, f'Corr: {corr:.4f}\nR²: {results.rsquared:.4f}',
               transform=axes[1, 1].transAxes, fontsize=10,
               verticalalignment='top', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))

plt.tight_layout()
plt.savefig('task8_fitted_values_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Fitted values analysis figure saved")

# ========== Step 8: White-Noise Tests ==========
print("\n" + "=" * 70)
print("[Step 8] Residual White-Noise Tests")
print("=" * 70)

# Shapiro–Wilk Normality
shapiro_stat, shapiro_p = stats.shapiro(residuals)
print("\n[Normality Test]")
print(f"  Shapiro–Wilk: stat={shapiro_stat:.6f}, p={shapiro_p:.6f}")
print("  → Residuals follow normality ✓" if shapiro_p > 0.05 else "  → Residuals deviate from normality")

# Ljung–Box
lb_test = acorr_ljungbox(residuals, lags=24, return_df=True)
print("\n[Ljung–Box White-Noise Test]")
print(f"  Lag 12: stat={lb_test.loc[12, 'lb_stat']:.4f}, p={lb_test.loc[12, 'lb_pvalue']:.6f}")
print(f"  Lag 24: stat={lb_test.loc[24, 'lb_stat']:.4f}, p={lb_test.loc[24, 'lb_pvalue']:.6f}")
print("  → Residuals are white noise ✓" if lb_test.loc[24, 'lb_pvalue'] > 0.05 else "  → Not white noise; autocorrelation remains")

# ACF significance
acf_values = acf(residuals, nlags=48, fft=False)
ci = 1.96/np.sqrt(len(residuals))
sig_lags = np.where(np.abs(acf_values[1:]) > ci)[0] + 1

print("\n[ACF Analysis]")
print(f"  Significant lags: {len(sig_lags)}/48")
if len(sig_lags) > 0:
    print(f"  Lags: {sig_lags[:15].tolist()}{'...' if len(sig_lags) > 15 else ''}")

# ========== Step 9: Summary ==========
print("\n" + "=" * 70)
print("[Step 9] Task 8 Summary")
print("=" * 70)

print(f"""
Question: Are there any remaining temporal components left?

[Effect of the Extended Harmonic Regression]

1) Model Improvement
   - Task 6 R² (annual only): {results_task6.rsquared:.4f}
   - Task 8 R² (annual + semiannual): {results.rsquared:.4f}
   - R² gain: {(results.rsquared - results_task6.rsquared):.4f}
   - Residual std: {results_task6.resid.std():.4f} → {residuals.std():.4f}
   - {"✓ Notable improvement" if results.rsquared > results_t_
